In [1]:
!wget -c https://www.openslr.org/resources/12/original-books.tar.gz
!test -e LibriSpeech || tar -xf original-books.tar.gz

--2022-04-23 10:14:00--  https://www.openslr.org/resources/12/original-books.tar.gz
Resolving www.openslr.org (www.openslr.org)... 64:ff9b::2e65:9e40
Connecting to www.openslr.org (www.openslr.org)|64:ff9b::2e65:9e40|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://openslr.elda.org/resources/12/original-books.tar.gz [following]
--2022-04-23 10:14:01--  http://openslr.elda.org/resources/12/original-books.tar.gz
Resolving openslr.elda.org (openslr.elda.org)... 2001:41d0:203:ad8a::
Connecting to openslr.elda.org (openslr.elda.org)|2001:41d0:203:ad8a::|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 297279345 (284M) [application/x-gzip]
Saving to: ‘original-books.tar.gz’

original-books.tar. 100%[===================>] 283.51M  54.8MB/s    in 5.6s    

2022-04-23 10:14:06 (51.1 MB/s) - ‘original-books.tar.gz’ saved [297279345/297279345]



In [2]:
import os
import nltk
from nltk.tokenize import sent_tokenize
import pandas as pd

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
MAX_LEN = 256*3  # 256 tokens of 3 char each
MAX_FILE = 10000000

In [4]:
def get_sentences(text):
    sentences = []
    for paragraph in text.split("\n\n"):
        paragraph = paragraph.replace("\n", " ").replace("  ", " ").replace("  ", " ").replace("  ", " ").strip()
        if paragraph:
            if len(paragraph) < MAX_LEN:
                sentences.append(paragraph)
            else:
                sentences.extend(sent_tokenize(paragraph))
    return sentences

def get_sentences_from_path(file_path):
    with open(file_path, "r") as f:
        return get_sentences(f.read(MAX_FILE))

In [5]:
all_sentences = []
for root, dirs, files in os.walk("LibriSpeech"):
    path = root.split(os.sep)
    for file in files:
        if file.endswith(".txt"):
            try:
                all_sentences.extend(get_sentences_from_path(root + "/" + file))
            except UnicodeDecodeError:
                pass
len(all_sentences)

4107551

In [6]:
df = pd.DataFrame(all_sentences, columns=["text"])
df.to_csv("gutenberg.tsv", sep="\t", index=None, header=None)